# ARIMA model on history travel time

In [2]:
%matplotlib inline
import lab.setup
import functools
import pandas as pd
import numpy as np
import numba

from pandas.tseries.offsets import *

import statsmodels.api as sm
import statsmodels.tsa.arima_model as arima_model

from statsmodels.graphics.api import qqplot

g_region_temporal = 5
g_region_spatial  = 1

DATA_PATH = 'dataset'

/home/mingyuan/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## load prepared data from linear.ipynb&& build features

In [3]:
ds_train_full = pd.read_csv('dataset/ds_filled_s1.csv', dtype={'link_ID':'uint64'}, low_memory=False)

In [4]:
ds_train_full.head(1)

,link_ID,time_intv,date,time_interval,travel_time,in_links,out_links,filled,uplink_0,uplink_1,uplink_2,uplink_3,downlink_0,downlink_1,downlink_2,downlink_3,uplink_mean_tt,downlink_mean_tt
0,3377906280028510514,2016-03-03 00:00:00,NaN,NaN,5.1,4377906282541600514,4377906280763800514,True,4377906282541600514,0,0,0,4377906280763800514,0,0,0,55.4,8.4


## with statsmodels pkg

In [5]:
TRAIN_SET_RATIO = 0.80

def get_series(df):
    s = df['travel_time']
    s.index = df['time_intv']
    s.index = pd.to_datetime(s.index)
    return s
    
link_no = ds_train_full.link_ID.unique().shape[0]
series_train = []
series_valid = []
counter = 0
for link_ID, link_ds in ds_train_full.groupby('link_ID'):
    counter += 1
    if link_ds.shape[0] != 720:
        print('!!!!!')
        
    if counter < link_no * TRAIN_SET_RATIO:
        series_train.append(get_series(link_ds))
    else:
        series_valid.append(get_series(link_ds))

In [9]:
import matplotlib.pyplot as plt
import warnings
import multiprocessing.pool as pool
import os

best_args = (3, 0, 4)

def mape(y_hat, y):
    """Compute root mean squared error"""
    return np.mean(((y - y_hat) / y).abs())

def train_multi_series(series, p, d, q):
    params = None
    running_loss = 0.0
    counter = 0
    for s in series:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            try:
                arma_mod = sm.tsa.ARIMA(s, order=(p, d, q))
                arma_res = arma_mod.fit(start_params=params)
            except Exception as e:
                # print(e)
                continue
                
            params = arma_res.params
            yhat = arma_res.predict(start='2016-03-03 01:00:00', end='2016-03-03 23:59:59', dynamic=False)
            loss = mape(yhat, s['2016-03-03 01:00:00':'2016-03-03 23:59:59'])
            
            running_loss += loss
            counter += 1
            
            if counter % 30 == 0:
                print('mean running loss:', running_loss / counter)
    
    # fig, ax = plt.subplots(figsize=(5,4))
    # fig = arma_res.plot_predict(start='2016-03-03 00:00:00', end='2016-03-04 02:00:00', ax=ax)
    # legend = ax.legend(loc='upper left')
    return running_loss / counter, arma_res

def validate(series, trained_params, p, d, q):
    running_loss = 0.0
    counter = 0
    for s in series:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            try:
                arma_mod = sm.tsa.ARIMA(s, order=(p, d, q))
                arma_mod.k_trend, arma_mod.exog = arima_model._make_arma_exog(s, None, 'c')
                arma_mod.nobs = s.shape[0]
                arma_mod.transparams = False
                # res = arma_mod.fit(start_params=trained_params)
            except Exception as e:
                raise
                continue
                
            yhat = arma_mod.predict(
                trained_params, start='2016-03-03 01:00:00', end='2016-03-03 23:59:59', dynamic=False)
            loss = mape(yhat, s['2016-03-03 01:00:00':'2016-03-03 23:59:59'])
            running_loss += loss
            counter += 1
    
    if counter == 0:
        # failed to converge
        return 1e6
        
    return running_loss / counter

def train_pqd(p, d, q):
    loss, arma_res = train_multi_series(series_train, p, d, q)
    valid_loss = validate(series_valid, arma_res.params, p, d, q)
    print("===== process:{}, qdp:{}, train-loss: {}, valid-loss: {} =====".format(
        os.getpid(), (p, d, q), loss, valid_loss))
    
    return loss, valid_loss, (p, d, q)

def grid_search():
    exec_pool = pool.Pool()
    results = []
    best = 1e6
    best_valid = 1e6
    for p in range(1, 5 + 1):
        for d in range(0, 1 + 1):
            for q in range(1, 5 + 1):
                results.append(exec_pool.apply_async(train_pqd, (p, d, q)))
    exec_pool.close()
    exec_pool.join()

    for result in results:
        r = result.get()
        valid_loss = r[1]
        if valid_loss < best_valid:
            best_valid = valid_loss
            best = r[0]
            best_args = r[2]

    print("===== best train-loss: {}, valid-loss: {} =====".format(best, best_valid))

train_pqd(3, 0, 4)

mean running loss: 0.2100010038474481
mean running loss: 0.20134923742857372
mean running loss: 0.20912543588405966
===== process:9841, qdp:(3, 0, 4), train-loss: 0.2092399314188365, valid-loss: 0.18407242805422236 =====


(0.2092399314188365, 0.18407242805422236, (3, 0, 4))

In [6]:
print(best, best_valid, best_args)

0.20959066377419935 0.1626550771151752 (3, 0, 4)
